In [1]:
import os
import pandas as pd

In [3]:
read_count=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/lc_20230512/lc_20230512_perind_numers.counts.gz",compression='gzip',sep=" ")


In [2]:
output_dir='/home/ls/rachelcw/projects/salmon/quant_results'
quant_files=[f for f in os.listdir(output_dir)  if f.endswith("_quant")]
sf_files=[f for f in quant_files for sf in os.listdir(os.path.join(output_dir,f)) if sf.endswith(".sf") ]
full_path_quant_files = [os.path.join(output_dir,f) for f in quant_files]


Agreggate the numReads for each sample to count read table

In [76]:
# List of sample names
# full_path_quant_files = [os.path.join(output_dir,f) for f in quant_files]

# Initialize an empty dictionary to store junction information
junctions_data = {}
parameters=['Name','Length','EffectiveLength','TPM','NumReads']
# Loop through each sample
for quant_file in full_path_quant_files:
    quant_file_sf = f"{quant_file}/quant.sf"  # Path to the quant.sf file for the sample
    
    # Read the quant.sf file into a Pandas DataFrame
    quant_data = pd.read_csv(quant_file_sf, sep="\t", usecols=[parameters[0], parameters[3]])
    # Extract the sample name from the file path
    sample_name = quant_file.split("/")[-1].split(".")[0]
    # Store the junction abundance in the dictionary
    junctions_data[sample_name] = quant_data.set_index(parameters[0])[parameters[3]]

# Create a Pandas DataFrame from the junctions_data dictionary
aggregated_table = pd.DataFrame(junctions_data)
aggregated_table.rename_axis("transcript_id", inplace=True)

# Save the aggregated table to a CSV file
# aggregated_table.to_csv("/home/ls/rachelcw/projects/salmon/all_sample_countreads_table.csv")


counts reads of GCLL samples from proteomics cohort

In [77]:
# table of only GCLL samples- columns name contain GCLL
gcll_table=aggregated_table[aggregated_table.columns[aggregated_table.columns.str.contains('GCLL')]]

In [78]:

gcll_proteomics=pd.read_csv("/data01/private/projects/splicing_cll/results/proteomics/all_sample_ids_concise - Sheet1.csv",usecols=["participant_id","dfci_sample_id"])
dict_gcll_proteomics=dict(zip(gcll_proteomics["dfci_sample_id"],gcll_proteomics["participant_id"]))

gcll_list=gcll_proteomics["participant_id"].tolist()

gcll_table_list=gcll_table.columns.tolist()


In [54]:
len(set(gcll_table_list))
len(set(gcll_list))

139

In [79]:
gcll_table.columns=[dict_gcll_proteomics[x] if x in dict_gcll_proteomics else x for x in gcll_table.columns]

In [80]:
# filter gcll table to only include samples that are in the proteomics data
# gcll_intersect=list(set(gcll_list).intersection(gcll_table_list))

gcll_proteomics_table=gcll_table[gcll_table.columns[gcll_table.columns.isin(gcll_list)]]
 


In [74]:
# all(gcll_proteomics_table.columns.isin(gcll_list))
gcll_proteomics_table.to_csv("/home/ls/rachelcw/projects/salmon/gcll_proteomics_TPM_table.csv")

In [37]:
# intersect between gcll list and gcll table list
gcll_intersect=list(set(gcll_list).intersection(gcll_table_list))


In [38]:
len(set(gcll_both))

119

In [23]:
# samples in proteomics but not in table-salmon quant results
gcll_proteomics_samples_not_quant=[s for s in gcll_list if s not in gcll_table_list]
# samples in proteomics and in table-salmon quant results
gcll_proteomics_samples_quant=[s for s in gcll_list if s in gcll_table_list]

In [30]:
gcll_new_bams=[f.split('.')[0] for f in os.listdir("/data01/private/data/cllmap/rnaseq/bams_cll11/") if f.endswith(".bam")]
gcll_new_bams_names=set([dict_gcll_proteomics[f] for f in gcll_new_bams if f in dict_gcll_proteomics.keys()])
gcll_proteomics_in_new_bams=[f for f in gcll_proteomics_samples_not_quant if f in gcll_new_bams_names]
# gcll samples in proteomics but dont have bam files
gcll_proteomics_no_bam=set([f for f in gcll_proteomics_samples_not_quant if f not in gcll_new_bams_names and f in gcll_list and f not in gcll_table_list ])
# with open("/home/ls/rachelcw/projects/salmon/gcll_proteomics_no_bam.txt", "w") as output:
#     output.write(",".join(gcll_proteomics_no_bam))


In [35]:
gcll_proteomics_bam_not_quant=set([f for f in gcll_new_bams_names if f in gcll_list and f not in gcll_table_list ])
with open("/home/ls/rachelcw/projects/salmon/gcll_proteomics_bam_not_quant.txt", "w") as output:
    output.write(",".join(gcll_proteomics_bam_not_quant))
# gcll_new_bams_names_quant=[f for f in gcll_new_bams_names if f in gcll_table_list]
# gcll_new_bams_names_not_quant=[f for f in gcll_new_bams_names if f not in gcll_table_list]

In [36]:
print(gcll_proteomics_bam_not_quant)

{'GCLL-0379', 'GCLL-0390', 'GCLL-0386', 'GCLL-0374', 'GCLL-0377', 'GCLL-0381', 'GCLL-0257', 'GCLL-0385', 'GCLL-0380', 'GCLL-0394', 'GCLL-0382', 'GCLL-0388', 'GCLL-0391', 'GCLL-0392', 'GCLL-0383', 'GCLL-0395', 'GCLL-0387', 'GCLL-0393', 'GCLL-0236', 'GCLL-0384', 'GCLL-0378', 'GCLL-0375', 'GCLL-0396', 'GCLL-0376'}


In [5]:
bam_files = [f for f in os.listdir(input_dir) if f.endswith(".bam")]
prefix_bam_files=[os.path.splitext(os.path.basename(f))[0].replace('.out','') for f in bam_files]
quant_files=[f for f in os.listdir(output_dir)  if f.endswith("_quant")]
sf_files=[f for f in quant_files for sf in os.listdir(os.path.join(output_dir,f)) if sf.endswith(".sf") ]
prefix_sf_files=[os.path.splitext(os.path.basename(f))[0] for f in sf_files]
list_samples=[f'{s}.out.bam' for s in prefix_bam_files if s not in prefix_sf_files ]

In [15]:
groupfile=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/groups_file.analysis.20230108/groups_file_a2.txt",sep="\t",header=None)

In [19]:
sf_files=[sf.split('.')[0] for sf in sf_files]

In [20]:
not_both_list= [item for item in samples if item not in sf_files]
print(not_both_list)

['GCLL-0004', 'GCLL-0055', 'GCLL-0065', 'GCLL-0100', 'GCLL-0103', 'GCLL-0119', 'GCLL-0142', 'GCLL-0144', 'GCLL-0146', 'GCLL-0169', 'GCLL-0191', 'GCLL-0293', 'SCLL-0471', 'GCLL-0021', 'GCLL-0052', 'GCLL-0159', 'GCLL-0166', 'GCLL-0180', 'GCLL-0190', 'GCLL-0200', 'GCLL-0294', 'GCLL-0299']


In [16]:
bam_files = [b for b in os.listdir(input_dir)]

In [15]:
bam_files = [f for f in os.listdir(input_dir) if f.endswith(".bam")]
output_prefix = os.path.splitext(os.path.basename(bam_files[-1]))[0]
fastq_prefix = os.path.join(output_dir, output_prefix)
if output_prefix not in fastq_done:
    print(output_prefix)

In [4]:
import pandas as pd
reference_file_path_mut="/data01/private/projects/splicing_cll/results/proteomics/analysis.20230713/mutated_cll_sf3b1_proteomics_reference.txt"
reference_file_path_unmut="/data01/private/projects/splicing_cll/results/proteomics/analysis.20230713/unmutated_cll_sf3b1_proteomics_reference.txt"
reference_file_mut=pd.read_csv(reference_file_path_mut,sep='\t',header=None,names=["chr","start","end","strand","ENST","ENSG"])
reference_file_unmut=pd.read_csv(reference_file_path_unmut,sep='\t',header=None,names=["chr","start","end","strand","ENST","ENSG"])
reference_file=pd.concat([reference_file_mut,reference_file_unmut])
reference_file.drop_duplicates(keep='first',inplace=True)
print(len(reference_file["ENST"].unique()))

463


In [17]:
import pysam

bam_path = "/data01/private/data/cllmap/data/rna/bams/DKFZ-CLL65.Aligned.sortedByCoord.out.bam"  # Replace with your BAM file path

# Open the BAM file
with pysam.AlignmentFile(bam_path, "rb") as bam_file:
    # for line in bam_file.head(10):
    #     print(line)
   
    # get header information
    # header = bam_file.header
    # print(header)
    # libraries = set()

    # # get transcriptome information
    # transcriptome = bam_file.references
    # print(transcriptome)
    print(bam_file.header.get("RG"))
    # # Iterate through the read groups in the header
    # for rg_dict in bam_file.header.get("RG", []):
    #     print(rg_dict)

    # # 
    # # Print the unique library IDs
    # for library in libraries:
    #     print(library)

[{'ID': 'rg1', 'SM': 'sm1'}]


In [8]:
transcripts=[]
with open("/home/ls/rachelcw/projects/salmon/transcriptome.fa", "r") as f:
    for line in f:
        line=line.strip()
        if line.startswith(">"):
            transcript=line.split("|")[1]
            transcripts.append(transcript)
# Read in the data


In [11]:
print(len(transcripts),len(set(transcripts)))

416 379
